In [1]:
import math
import numpy as np
from scipy import sparse
import pandas as pd
import copy
from datetime import datetime
from itertools import combinations
import os
import argparse
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pdb
import itertools
import time
import argparse
import warnings
import pickle
import dill

import sys
sys.path.append('..')

# from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_curve, auc, accuracy_score, confusion_matrix, roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from scipy.special import softmax, expit
from scipy.optimize import minimize, fmin_bfgs, fmin_tnc
from scipy import sparse
from absl import flags
from datetime import datetime
from itertools import combinations
from tqdm.auto import tqdm

from private_pgm_local.src.mbi import Dataset, FactoredInference
from diffprivlib_main.diffprivlib_local import models as dp
from private_pgm_local.mechanisms import aim
from dpsynth.workload import Workload

UTILS

In [2]:
def preprocess_data(dataset, target_dict, n_limit, train_ratio, one_hot):
    """
    Preprocesses the data for further analysis.

    Parameters:
        dataset (str): Name of the dataset.
        target_dict (dict): Dictionary containing target information for datasets.
        n_limit (int): Limit on the number of data points.
        train_ratio (float): Ratio of training data.
        one_hot (bool): Whether to perform one-hot encoding.

    Returns:
        X (df): train features
        X_test (df): test features
        y (df): train target
        y_test (df): test target
        pgm_train_df (df): data for training AIM (AIM processes raw categorical tabular data, pre encoding)
        domain (dict): attribute domain
        target (str): name of the chosen predicted variable
        attribute_dict (dict): attrinbute information in the form {'attr_name': [list of possible values]}
        features_to_encode (list): columns to encode
        encoded_features (list): list of feature names post encoding
        original_ranges (dict): dictionary of the attribute ranges in the form {'attr_name': [min, max]}
        training_columns (list): list of the features included in training phase
    """

    # Import the data
    csv_path = '../hd-datasets-master/clean/' + dataset + '.csv'
    meta_path = '../hd-datasets-master/clean/' + dataset + '-domain.json'
    data = Dataset.load(csv_path, meta_path)  # for PGM
    domain = data.domain
    target = target_dict[dataset]

    # Load the data
    df = pd.read_csv(csv_path)

    # Split the DF
    if len(df) > n_limit:
        df = df[:n_limit]
    X, y, X_test, y_test = splitdf(df, target, train_ratio)
    pgm_train_df = X.join(y)

    # Create dictionary with attribute levels
    attribute_dict = {}
    for col in df:
        unique_values = list(range(domain[col]))
        attribute_dict[col] = unique_values
    print(attribute_dict)

    # If one_hot is active, then we one hot both the train set and the test set.
    features_to_encode, training_columns = [], X.columns
    
    if one_hot:
        print(f"one-hot encoding {dataset}...")
        features_to_encode = get_features_to_encode(dataset)
        X_ohe = pd.get_dummies(X, columns=features_to_encode, drop_first=True)
        X_ohe.drop(X_ohe.std()[X_ohe.std() == 0].index, axis=1, inplace=True)
        for col in X_ohe:
            if col.endswith(".0"):
                X_ohe.rename(columns={col: col.split(".0")[0]}, inplace=True)
        training_columns = X_ohe.columns
        X_test_ohe = pd.get_dummies(X_test, columns=features_to_encode, drop_first=True)
        X_test_ohe = add_and_subsets_cols_to_test_set(X_test_ohe, training_columns)
        for col in X_test_ohe:
            if col.endswith(".0"):
                X_test_ohe.rename(columns={col: col.split(".0")[0]}, inplace=True)
        assert set(X_ohe.columns) == set(X_test_ohe.columns)
        X = X_ohe.copy(deep=True)
        X_test = X_test_ohe.copy()

    encoded_features = [col for col in X if col.split("_")[0] in features_to_encode]
    original_ranges = {feature: [0, domain[feature]] for feature in attribute_dict.keys()}

    if one_hot:
        X = normalize_minus1_1(X, attribute_dict, encoded_features)
        X_test = normalize_minus1_1(X_test, attribute_dict, encoded_features)
        y = normalize_minus1_1(y, attribute_dict, encoded_features)
        y_test = normalize_minus1_1(y_test, attribute_dict, encoded_features)

    return X, X_test, y, y_test, pgm_train_df, domain, target, attribute_dict, features_to_encode, encoded_features, original_ranges, training_columns


def splitdf(df, target, train_ratio):
    
    n = len(df)

    idxs = np.array(range(n))
    np.random.shuffle(idxs)

    train_rows, test_rows = idxs[:int(train_ratio * n)], idxs[int(train_ratio * n):]
    df_train, df_test = df.iloc[train_rows, :], df.iloc[test_rows, :]

    df_X_train, df_y_train = df_train.loc[:, df_train.columns != target], df_train.loc[:, df_train.columns == target]
    df_X_test, df_y_test = df_test.loc[:, df_test.columns != target], df_test.loc[:, df_test.columns == target]

    return (df_X_train, df_y_train, df_X_test, df_y_test)


def get_features_to_encode(dataset):
    if dataset == "adult":
        features_to_encode = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']
    elif dataset == "ACSincome" or "ACSincome-LIN":
        features_to_encode = ['COW', 'MAR', 'RELP', 'RAC1P']
    elif dataset == "ACSemployment":
        features_to_encode = ['MAR', 'RELP', 'CIT', 'MIL', 'ANC', 'RAC1P']
    elif dataset == "ACSmobility":
        features_to_encode = ['MAR', 'CIT', 'MIL', 'ANC', 'RELP', 'RAC1P', 'GCL', 'COW', 'ESR']
    elif dataset == "ACSPublicCoverage":
        features_to_encode = ['MAR', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC', 'ESR', 'FER', 'RAC1P']
    elif dataset == "ACSTravelTime":
        features_to_encode = ['MAR', 'ESP', 'MIG', 'RELP', 'RAC1P', 'CIT', 'OCCP', 'JWTR']
    elif dataset == "titanic":
        features_to_encode = ['Pclass', 'Cabin', 'Embarked']
    return features_to_encode


def add_and_subsets_cols_to_test_set(X_test, columns):
    for col_val in columns:
        if col_val not in X_test.columns:
            X_test[col_val] = 0
    return X_test[columns]


def selectTargetMarginals(cols, target, mode='target-pairs'):
    out = []
    if mode == 'target-pairs':
        for col in cols:
            if col != target:
                out.append((col, target))
    elif mode == 'target-triplets':
        cols_new = list(cols)
        cols_new.remove(target)
        tmp_pairs = combinations(cols_new, 2)
        out = [(t[0], t[1], target) for t in tmp_pairs]
    elif mode == 'target-pairs-target-triplets':
        out = []
        for col in cols:
            if col != target:
                out.append((col, target))
        cols_new = list(cols)
        cols_new.remove(target)
        tmp_pairs = combinations(cols_new, 2)
        out.extend([(t[0], t[1], target) for t in tmp_pairs])
    elif mode == "all-pairs":
        out = list(combinations(cols, 2))
    elif mode == 'all-triplets':
        out = list(combinations(cols, 3))
    elif mode == "no-target-pairs":
        cols_new = list(cols)
        cols_new.remove(target)
        out = combinations(cols_new, 2)
    return out

def get_bound_XTX(attribute_dict, target, features_to_encode, one_hot):

    if not one_hot: # then data is binary synthetic data
        bound_X = np.sqrt(np.sum([max(attribute_dict[f])**2 for f in attribute_dict if f!=target]))
        bound_XTX = bound_X**2
        
    elif one_hot: # follow sensitivity computation as described in paper
        bound_XTX = len(attribute_dict.keys()) - 1    #excludes target
        bound_X = np.sqrt(bound_XTX)

    return bound_XTX, bound_X

def normalize_minus1_1(X, attribute_dict, encoded_features):
    original_ranges = {attr: [min(attribute_dict[attr]), max(attribute_dict[attr])]
                       for attr in attribute_dict.keys()}

    X_out = pd.DataFrame()

    for col in X.columns:

        # this is in case the test set has columns of zeros
        if len(set(X[col])) == 1:
            X_out[col] = X[col]

        # if the column corresponds to a categorical feature that has been one-hot encoded, keep it in domain [0, 1]
        if col in encoded_features:
            X_out[col] = X[col]

        # for all other features, rescale to [-1, 1]
        else:
            colmin = original_ranges[col][0]
            colmax = original_ranges[col][1]

            col_1s = (1 - (-1)) * ((X[col] - colmin) / (colmax - colmin)) - 1
            X_out[col] = col_1s

    return X_out


# AIM Utils ############################################################################################################

class PGMsynthesizer():

    # this class is a wrapper for constructing the AIM model

    def __init__(self, data, epsilon, delta, measurements, model_size, max_iters, num_synth_rows):
        self.epsilon = epsilon
        self.delta = delta
        self.data = data
        self.measurements = measurements
        self.model_size = model_size
        self.max_iters = max_iters
        self.num_synth_rows = num_synth_rows
        self.aim_model = None
        self.synth = None
        self.G = None
        self.ans_wkld = None

    def mstsynthesizer(self):
        self.synth, self.G = mst.mst(self.data, self.epsilon, self.delta)

    def mwemsynthesizer(self):
        self.synth, self.G = mwem.mwem_pgm(self.data, self.epsilon, workload=self.measurements)

    def v13synthesizer(self):
        v13model = v13.V13(self.epsilon, self.delta)
        self.synth, self.G = v13model.run(self.data, self.measurements)

    def aimsynthesizer(self):
        aim_model = aim.AIM(epsilon=self.epsilon, delta=self.delta, max_iters=self.max_iters,
                           max_model_size=self.model_size)
        self.synth, self.G, self.ans_wkld = aim_model.run(self.data, self.measurements, self.num_synth_rows,
                                                         output_graphical_model=True)


# DQ Query Approximation Utils #########################################################################################

def expand_W(W, attribute_dict):
    # add symmetric entries
    W_expanded = copy.deepcopy(W)
    for el in W:
        W_expanded[el[1], el[0]] = W[el].T

    # add (x, x) pairs
    for col in attribute_dict.keys():
        table_with_col = [W_expanded[tple] for tple in W_expanded if tple[0] == col][0]
        col_counts = np.sum(table_with_col, axis=1)
        W_expanded[col, col] = np.diag(col_counts)

    return W_expanded

class Chebyshev:
    """
    Chebyshev(a, b, n, func)
    Given a function func, lower and upper limits of the interval [a,b],
    and maximum degree n, this class computes a Chebyshev approximation
    of the function.
    Method eval(x) yields the approximated function value.
    """

    def __init__(self, a, b, n, func):
        self.a = a
        self.b = b
        self.func = func

        bma = 0.5 * (b - a)
        bpa = 0.5 * (b + a)
        f = [func(math.cos(math.pi * (k + 0.5) / n) * bma + bpa) for k in range(n)]
        fac = 2.0 / n
        self.c = [fac * sum([f[k] * math.cos(math.pi * j * (k + 0.5) / n)
                             for k in range(n)]) for j in range(n)]

    def eval(self, x):
        a, b = self.a, self.b
        y = (2.0 * x - a - b) * (1.0 / (b - a))
        y2 = 2.0 * y
        (d, dd) = (self.c[-1], 0)  # Special case first step for efficiency
        for cj in self.c[-2:0:-1]:  # Clenshaw's recurrence
            (d, dd) = (y2 * d - dd + cj, d)
        return y * d - dd + 0.5 * self.c[0]  # Last step is different


def phi_logit(x):
    return -math.log(1 + math.exp(-x))

def logit_2(x):
    return math.log(1 + math.exp(x))


def get_ZTZ(W, attribute_dict, columns, features_to_encode, rescale):
    """
        W: [dict] marginal tables in the form {("feature_A", "feature_B"); m_A x m_B np.array of counts, ...}
        attribute_dict: [dict] attribute levels, {"attr_A": list of ordered possible levels for attr_A, ...}
                                - should include target
        columns: [list] list of names of post-encoding attributes
        features_to_encode: [list] list of features requiring 1-hot encoding}
        rescale: [boolean] True if rescaling numerical non binary attributes in [-1, 1], TBC
    """

    # initialize ZTZ as a DataFrame with *named* columns and rows
    base_matrix = np.zeros((len(columns), len(columns)))
    ZTZ = pd.DataFrame(base_matrix, columns=columns, index=columns)

    # loop through attribute pairs

    for a, attr_a in enumerate(columns):
        for b, attr_b in enumerate(columns[a:]):

            # root name of the attributes
            attr_a_orig = attr_a.split("_")[0]
            attr_b_orig = attr_b.split("_")[0]
            # possible level values
            a_values = attribute_dict[attr_a_orig]
            b_values = attribute_dict[attr_b_orig]
            if rescale:
                if attr_a_orig not in features_to_encode:
                    a_range_min, a_range_max = min(a_values), max(a_values)
                    a_values = [(1 -(-1)) * ((val - a_range_min) / (a_range_max - a_range_min)) - 1 for val in a_values]
                if attr_b_orig not in features_to_encode:
                    b_range_min, b_range_max = min(b_values), max(b_values)
                    b_values = [(1 -(-1)) * ((val - b_range_min) / (b_range_max - b_range_min)) - 1 for val in b_values]

            # case 1: a and b are both ordinal
            if attr_a_orig not in features_to_encode and attr_b_orig not in features_to_encode:
                mu_ab = W[(attr_a_orig, attr_b_orig)]
                for j, j_value in enumerate(a_values):
                    for k, k_value in enumerate(b_values):
                        ZTZ[attr_a][attr_b] += j_value * k_value * mu_ab[j, k]

            # case 2.1: a is ordinal, b is encoded
            elif attr_a_orig not in features_to_encode and attr_b_orig in features_to_encode:
                mu_ab = W[(attr_a_orig, attr_b_orig)]
                t = int(float(attr_b.split("_")[-1]))  # get level number ***** ASSUMES LEVELS CORRESPOND TO THE NAMES ******
                ZTZ[attr_a][attr_b] = np.sum(np.multiply(mu_ab[:, t], a_values))

            # case 2.2: a is encoded, b is ordinal
            elif attr_a_orig in features_to_encode and attr_b_orig not in features_to_encode:
                mu_ab = W[(attr_a_orig, attr_b_orig)]
                s = int(float(attr_a.split("_")[-1]))  # get level number ***** ASSUMES LEVELS CORRESPOND TO THE NAMES ******
                ZTZ[attr_a][attr_b] = np.sum(np.multiply(mu_ab[s, :], b_values))

            # case 3: a and b are both encoded
            elif attr_a_orig in features_to_encode and attr_b_orig in features_to_encode:
                s = int(float(attr_a.split("_")[-1]))  # get level number ***** ASSUMES LEVELS CORRESPOND TO THE NAMES ******
                t = int(float(attr_b.split("_")[-1])) # get level number ***** ASSUMES LEVELS CORRESPOND TO THE NAMES ******
                mu_ab = W[(attr_a_orig, attr_b_orig)]
                ZTZ[attr_a][attr_b] = mu_ab[s, t]

    # copy lower tri to upper tri
    ZTZ = ZTZ + ZTZ.T - np.diag(np.diag(ZTZ))

    return ZTZ

LOGREG UTILS

In [3]:
class LogisticRegressionObjective():
    @staticmethod
    def sigmoid_v2(x, theta):
        z = np.dot(x, theta)
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def hypothesis(theta, x):
        return LogisticRegressionObjective.sigmoid_v2(x, theta)

    @staticmethod
    def loss(theta, x, y):
        m = x.shape[0]
        h = LogisticRegressionObjective.hypothesis(theta, x)
        return -(1 / m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))

    @staticmethod
    def gradient(theta, x, y):
        m = x.shape[0]
        h = LogisticRegressionObjective.hypothesis(theta, x)
        return (1 / m) * np.dot(x.T, (h - y))

def genobjpert_get_params(X, epsilon, delta, lmda, zeta):

    n, d = X.shape

    delta_thrs = 2*lmda/epsilon
    Delta = delta_thrs
    b_var = zeta**2 * (8 * np.log(2/delta) + 4*epsilon) / (epsilon**2) * np.eye(d)
    b = np.random.multivariate_normal(np.zeros((b_var.shape[0], )), b_var)

    return Delta, b

def dp_objective(theta, X, y, n, d, Delta, b):

    base_loss = LogisticRegressionObjective.loss(theta, X, y)
    regularizer = 1/n * 0   #assumed zero
    sec_term = Delta/(2*n) * np.dot(theta.T, theta)
    third_term = np.dot(b.T, theta)/n

    return base_loss + regularizer + sec_term + third_term

def dp_gradient(theta, X, y, n, d, Delta, b):
    base_gradient = LogisticRegressionObjective.gradient(theta, X, y)

    reg_term = 1/n * np.zeros((d,))  # Assumed zero
    second_term = Delta/n * theta
    third_term = b/n

    return base_gradient + reg_term + second_term + third_term

def get_dp_approx_ll(theta, yTX, XTXy2, a, b, c, n):
    dp_approx_ll = n * a + np.dot(theta, yTX) * b + np.dot(np.dot(theta, XTXy2), theta) * c
    return dp_approx_ll

class SSApproxLL():

    def __init__(self, ch, yTX, XTXy2, n, penalty, alpha):
        self.n = n
        self.ch = ch
        self.penalty = penalty
        self.alpha = alpha
        self.theta = None
        self.yTX = yTX
        self.XTXy2 = XTXy2

    def fit(self):
        self.optimize()
        return self

    def log_likelihood(self, theta):
        a, b, c = self.ch.c
        term = get_dp_approx_ll(theta, self.yTX, self.XTXy2, a, b, c, self.n)
        term = 1 / self.n * term
        return term

    def optimize(self):

        def l2_penalty(theta):
            return np.sum(theta ** 2)

        x0 = [.0] * len(self.yTX)

        if self.penalty == None:
            res = minimize(lambda theta: -self.log_likelihood(theta),
                           x0,
                           method='L-BFGS-B',
                           options={'maxiter': 10000},
                           tol=0.00001)
            theta_star = res.x
            fun = res.fun

        elif self.penalty == 'l2':
            res = minimize(lambda theta: -self.log_likelihood(theta) + self.alpha * l2_penalty(theta),
                           x0,
                           method='L-BFGS-B',
                           options={'maxiter': 10000},
                           tol=0.00001)
            theta_star = res.x
            fun = res.fun - self.alpha * l2_penalty(theta_star)

        else:
            raise ValueError('Unknown penalty type, choose None or l2')

        self.theta = theta_star

    def predict_proba(self, X):
        z = np.dot(X, self.theta.T)
        y_pred_proba = expit(z)
        return (y_pred_proba)

    def predict(self, X, threshold=0.5):
        z = np.dot(X, self.theta.T)
        y_pred_proba = expit(z)
        y_pred = 2 * ((y_pred_proba >= threshold).astype(int)) - 1
        print("y pred dpapproxss", y_pred)
        return (y_pred)

def get_aim_model(pgm_train_df, domain, target, marginals_pgm, epsilon, delta, model_size, max_iters, n_samples):
    pgm_dataset = Dataset(pgm_train_df, domain)
    mrgs = selectTargetMarginals(pgm_train_df.columns, target, mode=marginals_pgm)
    mrgs_wkld = Workload((mrg, sparse.identity) for mrg in mrgs)
    pgm = PGMsynthesizer(pgm_dataset, epsilon, delta, mrgs_wkld, model_size, max_iters, n_samples)
    pgm.aimsynthesizer()
    return pgm, mrgs

def testLogReg(theta, X_test, y_test):
    logits = np.dot(X_test, theta)
    probabilities = 1 / (1 + np.exp(-logits))
    auc = roc_auc_score(y_test, probabilities)
    return auc

LOGRER METHODS

In [4]:
def public_logreg(X, y):
    model = LogisticRegression(penalty=None, fit_intercept=False, max_iter=1000)
    model.fit(X.to_numpy(), y.to_numpy().ravel())
    theta = model.coef_.ravel()   # probabilities for class 1
    theta = pd.DataFrame(theta)
    theta.set_index(X.columns, inplace=True)
    return theta

def dp_query_approx_ss_logreg(W_expanded, attribute_dict, training_columns, encoded_features, target, domain, n,
                              features_to_encode, one_hot, cheb, phi_logit, C=1.0):

    all_attributes_expanded = training_columns.append(pd.Index([target]))
    ZTZ = get_ZTZ(W_expanded, attribute_dict, all_attributes_expanded, features_to_encode, rescale=True)
    XTXy2 = ZTZ.loc[training_columns, training_columns]
    XTy = ZTZ.loc[training_columns, target]
    alpha = 1 / (n * C)
    model = SSApproxLL(cheb, XTy, XTXy2, n, penalty=None, alpha=alpha)
    model.fit()
    theta = model.theta

    return theta

def objective_perturbation_method(X, y, attribute_dict, target, features_to_encode, epsilon, delta, one_hot):

    n, d = X.shape
    # maxvals = np.array(np.amax(X.to_numpy(), axis = 0)).reshape(-1,1)
    # max_row_norm = np.sqrt(np.sum([a**2 for a in maxvals]))
    if one_hot:
        max_row_norm = np.sqrt(len(attribute_dict.keys())-1)   # excludes target
    else:
        _, max_row_norm = get_bound_XTX(attribute_dict, target, features_to_encode, one_hot)
    bound_y = np.abs(max(attribute_dict[target]))

    zeta = max_row_norm
    lmda = smooth_const = max_row_norm ** 2 / 4

    def sigmoid_v2(x, theta):
        z = np.dot(x, theta)
        return 1 / (1 + np.exp(-z))

    def hypothesis(theta, x):
        return sigmoid_v2(x, theta)

    def cost_function(theta, x, y):
        m = x.shape[0]
        h = hypothesis(theta, x)
        return -(1 / m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))

    def gradient_fun(theta, x, y):
        m = x.shape[0]
        h = hypothesis(theta, x)
        return (1 / m) * np.dot(x.T, (h - y))

    # Run the object perturbation
    Delta, b = genobjpert_get_params(X.to_numpy(), epsilon, delta, lmda, zeta)

    # Run the iteration
    # fmin_tnc returns the convergence message as third argument in its output
    # 0 means local minimium reached, 1 and 2 means convergence by function value or theta value
    # 3 is maximum number of iterations reached, 4 linear search failed
    finish_opt = False
    patience = 3

    while not finish_opt and patience > 0:

        theta0 = np.random.normal(loc=0, scale=0.01, size=X.shape[1]).reshape(-1, )
        theta_opt = fmin_tnc(func=dp_objective, x0=theta0, fprime=dp_gradient, maxfun=10000, disp=0,
                             args=(X.to_numpy(), y.to_numpy().reshape(-1, ), n, d, Delta, b))

        theta_final, n_it_run, final_message = theta_opt

        if final_message in [0, 1, 2]:
            finish_opt = True
        else:
            patience -= 1

    return theta_final

## LOGISTIC REGRESSION

In [33]:
dataset = 'adult'
csv_path = '../hd-datasets-master/clean/' + dataset + '.csv'
meta_path = '../hd-datasets-master/clean/' + dataset + '-domain.json'
epsilon = 0.1
delta = 1e-5
n_limit = 20_000
train_ratio = 0.7
one_hot = True

# AIM model parameters
model_size = 100  
max_iters = 1000  
PGMmarginals = 'all-pairs'

target_dict = {'adult': 'income>50K', 'titanic': 'Survived', 'diabetes': 'Outcome',
               'ACSemployment': 'ESR', 'ACSincome': 'PINCP', "ACSmobility": 'MIG',
               "ACSPublicCoverage": 'PUBCOV', 'ACSTravelTime': 'JWMNP', 'logregbinary10': 'predicted',
               'logregbinary5lowcorr': 'predicted', 'logregbinary7lowcorr': 'predicted',
               'logregbinary10lowcorr': 'predicted', 'logregbinary20lowcorr': 'predicted',
               'logregbinary30lowcorr': 'predicted'}

np.random.seed(237)

(X, X_test, y, y_test, 
 pgm_train_df, domain, target, 
 attribute_dict, features_to_encode, encoded_features, 
 original_ranges, training_columns) = preprocess_data(dataset, target_dict, n_limit, train_ratio, one_hot)

n, d = X.shape

print(f"X.shape {X.shape}")
print(f"X_test.shape {X_test.shape}")
print(f"y.shape {y.shape}")
print(f"y_test.shape {y_test.shape}")

{'age': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], 'workclass': [0, 1, 2, 3, 4, 5, 6, 7, 8], 'fnlwgt': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], 'education': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'education-num': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'marital-status': [0, 1, 2, 3, 4, 5, 6], 'occupation': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'relationship': [0, 1, 2, 3, 4, 5], 'race': [0, 1, 2, 3, 4], 'sex': [0, 1], 'capital-gain': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], 'capital-loss': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], 'hours-per-week': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23

PUBLIC LOGISTIC REGRESSION

In [34]:
theta_public = public_logreg(X, y)

TRAIN AND SAVE AIM MODEL, OBTAIN MARGINAL TABLES AND SYNTHETIC DATA

In [ ]:
# 1) get AIM model and save it
aim_model, workload = get_aim_model(pgm_train_df, domain, target, PGMmarginals, epsilon, delta, model_size, max_iters, len(X))
aim_model_graph = aim_model.G
with open('aim_model.pkl', 'wb') as f:
    dill.dump(aim_model_graph, f)

# 2) load AIM model and get marginal tables and synthetic data X_synth, y_synth
with open('aim_model.pkl', 'rb') as f:
    aim_model_graph = dill.load(f)

aim_ans_wkld = {cl: aim_model_graph.project(cl) for cl in workload}
W = {key: aim_ans_wkld[key].__dict__['values'] for key in aim_ans_wkld}

synth = aim_model_graph.synthetic_data(rows=n).df
synth_X, synth_y = synth.loc[:, synth.columns != target], synth.loc[:, synth.columns == target]

Initial Sigma 554.9176837222757
(!!!!!!!!!!!!!!!!!!!!!!) Reducing sigma 277.45884186113784


DPQUERYSS

In [ ]:
W_expanded = expand_W(W, attribute_dict)
train_columns = X.columns

# get Chebyshev
r = 6
degree = 3
C = 1.0
cheb = Chebyshev(-r, r, degree, phi_logit)

# approximate sufficient statistics
theta_dpqueryss = dp_query_approx_ss_logreg(W_expanded, attribute_dict, training_columns, 
                                            encoded_features, target, domain, n,
                                            features_to_encode, one_hot, cheb, phi_logit, C)

DP AIM SYNTHETIC DATA

In [ ]:
synth_X_ohe = pd.get_dummies(synth_X, columns=features_to_encode, drop_first=True)

missing_cols = []
for col in X.columns:
    if col not in synth_X_ohe.columns:
        missing_cols.append(col)

theta_aimsynth = public_logreg(synth_X_ohe, synth_y).T
for col in missing_cols:
    theta_aimsynth[col] = 0

theta_aimsynth = theta_aimsynth.reindex(columns=X.columns, fill_value=0)

for i,col in enumerate(X.columns):
    assert col == theta_aimsynth.columns[i] 

theta_aimsynth = theta_aimsynth.T.to_numpy()

DP OBJECTIVE PERTURBATION

In [ ]:
theta_objpert = objective_perturbation_method(X, y, attribute_dict, target, features_to_encode, epsilon, delta, one_hot)

EVALUATION

In [ ]:
auc_public = testLogReg(theta_public, X_test, y_test)
auc_dpqueryss = testLogReg(theta_dpqueryss, X_test, y_test)
auc_aimsynth = testLogReg(theta_aimsynth, X_test, y_test)
auc_aimsynth = testLogReg(theta_aimsynth, X_test, y_test)
auc_objpert = testLogReg(theta_objpert, X_test, y_test)

print(f"public auc: {auc_public}")
print(f"dpqueryss auc: {auc_dpqueryss}")
print(f"aimsynth auc: {auc_aimsynth}")
print(f"objpert auc: {auc_objpert}")